In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jpx-tokyo-stock-exchange-prediction/stock_list.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/sample_submission.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/options.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/financials.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/secondary_stock_prices.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/trades.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/stock_prices.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/jpx_tokyo_market_prediction/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/jpx-tokyo-stock-exchange-prediction/jpx_tokyo_market_prediction/__init__.py
/kaggle/input/jpx-tokyo-stock-exchange-prediction/data_specifications/stock_fin_spec.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/data_specifications/trades_spec.csv
/kaggle/input/jpx-tokyo-stock-

In [2]:
%time 
df = pd.read_csv("/kaggle/input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv")
print(df.shape)
df.head()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.15 µs
(269881, 12)


,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20211206_1301,2021-12-06,1301,2982.0,2982.0,2965.0,2971.0,8900,1.0,NaN,False,-0.003263
1,20211206_1332,2021-12-06,1332,592.0,599.0,588.0,589.0,1360800,1.0,NaN,False,-0.008993
2,20211206_1333,2021-12-06,1333,2368.0,2388.0,2360.0,2377.0,125900,1.0,NaN,False,-0.009963
3,20211206_1375,2021-12-06,1375,1230.0,1239.0,1224.0,1224.0,81100,1.0,NaN,False,-0.015032
4,20211206_1376,2021-12-06,1376,1339.0,1372.0,1339.0,1351.0,6200,1.0,NaN,False,0.002867


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 269881 entries, 0 to 269880
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   RowId             269881 non-null  object 
 1   Date              269881 non-null  object 
 2   SecuritiesCode    269881 non-null  int64  
 3   Open              269063 non-null  float64
 4   High              269063 non-null  float64
 5   Low               269063 non-null  float64
 6   Close             269063 non-null  float64
 7   Volume            269881 non-null  int64  
 8   AdjustmentFactor  269881 non-null  float64
 9   ExpectedDividend  2011 non-null    float64
 10  SupervisionFlag   269881 non-null  bool   
 11  Target            269873 non-null  float64
dtypes: bool(1), float64(7), int64(2), object(2)
memory usage: 22.9+ MB


In order to rank with LightGBM Ranker, we need to form some groups up to 31. This means that; actually the model will try to learn in whic group an id is belong to..

In [4]:
### drop the rows with target as null values and take it as test data
test_data = df[df["Target"].isna()]
df = df.dropna(subset=["Target"])

In [5]:
df.groupby("Date")["Target"].rank("dense", ascending=False).isna().any()

False

In [6]:
df["Target"] = df.groupby("Date")["Target"].rank("dense", ascending=False).astype(int)
df.head()

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20211206_1301,2021-12-06,1301,2982.0,2982.0,2965.0,2971.0,8900,1.0,NaN,False,1286
1,20211206_1332,2021-12-06,1332,592.0,599.0,588.0,589.0,1360800,1.0,NaN,False,1549
2,20211206_1333,2021-12-06,1333,2368.0,2388.0,2360.0,2377.0,125900,1.0,NaN,False,1591
3,20211206_1375,2021-12-06,1375,1230.0,1239.0,1224.0,1224.0,81100,1.0,NaN,False,1720
4,20211206_1376,2021-12-06,1376,1339.0,1372.0,1339.0,1351.0,6200,1.0,NaN,False,948


In [7]:
df["Target"] = pd.qcut(df.Target, 30).cat.codes

In [8]:
df.head()

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20211206_1301,2021-12-06,1301,2982.0,2982.0,2965.0,2971.0,8900,1.0,NaN,False,20
1,20211206_1332,2021-12-06,1332,592.0,599.0,588.0,589.0,1360800,1.0,NaN,False,25
2,20211206_1333,2021-12-06,1333,2368.0,2388.0,2360.0,2377.0,125900,1.0,NaN,False,25
3,20211206_1375,2021-12-06,1375,1230.0,1239.0,1224.0,1224.0,81100,1.0,NaN,False,27
4,20211206_1376,2021-12-06,1376,1339.0,1372.0,1339.0,1351.0,6200,1.0,NaN,False,15


In [9]:
print(df.Date.agg(['min', 'max']))

min    2021-12-06
max    2022-06-24
Name: Date, dtype: object


In [10]:
# Just some arbitrary dates
time_config = {'train_split_date': '2021-12-06',
               'val_split_date'  : '2022-02-10',
               'test_split_date' : '2022-02-20'}

train = df[(df.Date >= time_config['train_split_date']) & (df.Date < time_config['val_split_date'])]
val = df[(df.Date >= time_config['val_split_date']) & (df.Date < time_config['test_split_date'])]
test = df[(df.Date >= time_config['test_split_date'])]

print(train.shape)
print(val.shape)
print(test.shape)

col_use = [c for c in df.columns if c not in ["RowId","Date", "Target"]]

(90000, 12)
(12000, 12)
(167873, 12)


In [11]:
query_train = [train.shape[0] /2000] * 2000 #Because we have 2000 stock in each time group
query_val = [val.shape[0] / 2000] * 2000
query_test = [test.shape[0] / 2000] *2000

In [12]:
train[col_use]

,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag
0,1301,2982.0,2982.0,2965.0,2971.0,8900,1.0,NaN,False
1,1332,592.0,599.0,588.0,589.0,1360800,1.0,NaN,False
2,1333,2368.0,2388.0,2360.0,2377.0,125900,1.0,NaN,False
3,1375,1230.0,1239.0,1224.0,1224.0,81100,1.0,NaN,False
4,1376,1339.0,1372.0,1339.0,1351.0,6200,1.0,NaN,False
...,...,...,...,...,...,...,...,...,...
89995,9990,519.0,520.0,514.0,516.0,43300,1.0,NaN,False
89996,9991,810.0,819.0,806.0,819.0,15200,1.0,NaN,False
89997,9993,1648.0,1655.0,1645.0,1655.0,4700,1.0,NaN,False
89998,9994,2503.0,2509.0,2500.0,2500.0,6300,1.0,NaN,False


In [13]:
train['Target']

0        20
1        25
2        25
3        27
4        15
         ..
89995     1
89996    10
89997    18
89998     1
89999    17
Name: Target, Length: 90000, dtype: int8

In [14]:
from lightgbm import LGBMRanker

model_return = LGBMRanker(n_estimators=15000,
                          random_state=42,
                          num_leaves=41,
                          learning_rate=0.002,
                          #max_bin =20,
                          #subsample_for_bin=20000,
                          colsample_bytree=0.7,
                          n_jobs=2)
model_return.fit(train[col_use], train['Target'],
             group = query_train,
             verbose=100,
             early_stopping_rounds=200,
             eval_set=[(val[col_use], val['Target'])],
             eval_group=[query_val],
             eval_at=[1] #Make evaluation for target=1 ranking, I choosed arbitrarily
                )

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	valid_0's ndcg@1: 0.275999
[200]	valid_0's ndcg@1: 0.273488


LGBMRanker(colsample_bytree=0.7, learning_rate=0.002, n_estimators=15000,
           n_jobs=2, num_leaves=41, random_state=42)

In [15]:
test["pred"] = model_return.predict(test[col_use])
test["pred"] # So our output is not ranks, yet..

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


102000   -0.013337
102001    0.000937
102002   -0.011675
102003   -0.009435
102004   -0.003522
            ...   
269876   -0.019247
269877   -0.019247
269878   -0.006081
269879   -0.017064
269880   -0.011523
Name: pred, Length: 167873, dtype: float64

In [16]:
# load Time Series API
import jpx_tokyo_market_prediction
# make Time Series API environment (this function can be called only once in a session)
env = jpx_tokyo_market_prediction.make_env()
# get iterator to fetch data day by day
iter_test = env.iter_test()

In [17]:
for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    try:
        sample_prediction['Rank'] = model_return.predict(prices[col_use]) * -1
        # Get the ranks from prediction first and for the duplicated ones, just rank again
        sample_prediction['Rank'] = sample_prediction.groupby("Date")["Rank"].rank("dense", 
                                                                                   ascending=False).astype(int)
        sample_prediction['Rank'] = sample_prediction.groupby("Date")["Rank"].rank("first").astype(int) - 1
    except:
        sample_prediction['Rank'] = 0
    sample_prediction = sample_prediction.replace([-np.inf, np.inf], np.nan).fillna(0.0)
    # register your predictions
    env.predict(sample_prediction)
    display(sample_prediction)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


,Date,SecuritiesCode,Rank
0,2021-12-06,1301,448
1,2021-12-06,1332,1950
2,2021-12-06,1333,481
3,2021-12-06,1375,814
4,2021-12-06,1376,907
...,...,...,...
1995,2021-12-06,9990,154
1996,2021-12-06,9991,155
1997,2021-12-06,9993,858
1998,2021-12-06,9994,110


,Date,SecuritiesCode,Rank
0,2021-12-07,1301,449
1,2021-12-07,1332,1808
2,2021-12-07,1333,417
3,2021-12-07,1375,733
4,2021-12-07,1376,842
...,...,...,...
1995,2021-12-07,9990,130
1996,2021-12-07,9991,131
1997,2021-12-07,9993,783
1998,2021-12-07,9994,189


### Basic on quartile using pd.qcut

In [27]:
a = np.array([1, 2, 3, 4, 5]) * 100
a

array([100, 200, 300, 400, 500])

In [28]:
pd.qcut(a, 4)


[(99.999, 200.0], (99.999, 200.0], (200.0, 300.0], (300.0, 400.0], (400.0, 500.0]]
Categories (4, interval[float64, right]): [(99.999, 200.0] < (200.0, 300.0] < (300.0, 400.0] < (400.0, 500.0]]